How to convert a Google sheet to a static marketing website

templates google doc?

templates google sheet?

In [ ]:
var util = require('util');
var GoogleSpreadsheet = require('google-spreadsheet');

function getTemplates(doc) {
    if(typeof doc === 'string') {
        doc = new GoogleSpreadsheet(doc);
    }
    return util.promisify(doc.getInfo)()
        .then(info => {
            console.log('Loaded doc: ' + info.title + ' by ' + info.author.email);
            return info;
        })
        .then(s => s.worksheets.reduce((obj, worksheet) => {
            var layoutTitle = worksheet.title.toLowerCase()
                .replace(/\s*layout|\s*data|\s*template/ig, '')
                .split(/\s+/ig).join('-');
        
            if( typeof obj[layoutTitle] == 'undefined' ) obj[layoutTitle] = {template: null, data: null};
            if( worksheet.title.toLowerCase().includes('data') ) obj[layoutTitle].data = worksheet;
            else obj[layoutTitle].template = worksheet;
            return obj;
        }, {}))
}
module.exports = getTemplates;


test templates google sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get templates from sheet', () => {
    
    it('should have at least one page', () => {
        return getTemplates(docsId)
            .then(t => {
                console.log(JSON.stringify(Object.keys(t).reduce((obj, cur) => {
                    obj[cur] = {};
                    obj[cur].template = ((t[cur].template || {}).title || '').substr(0, 1000);
                    obj[cur].data = (t[cur].data ? ('yes: ' + t[cur].data.rowCount) : 'no');
                    return obj;
                }, {}), null, 4))
                assert(Object.keys(t).length > 0, 'should have a page called template');
            })
    })
    
    
})


get google data sheet?

google sheet array objects?


In [ ]:
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});
var util = require('util');

function getDataSheet(worksheet) {
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => {
            var columns = [];
            return cells.reduce((acc, o, arr) => {
                if (o.row == 1) {
                    columns[o.col] = o.value;
                    return acc;
                }
                if (!acc[o.row - 2]) {
                    acc[o.row - 2] = {};
                }
                var html = md.render(o.value);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(html.replace(/<\/?p>|\s+/ig, '') == o.value.replace(/<\/?p>|\s+/ig, '')) {
                    html = o.value;
                }
                acc[o.row - 2][columns[o.col]] = html;
                return acc;
            }, []);
        })
}
module.exports = getDataSheet;


test google data sheet?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get data from data sheet', () => {
    
    it('should process at least one data sheet', () => {
        return getTemplates(docsId)
            .then(t => {
                var key = Object.keys(t).filter(k => t[k].data)[0];
                assert(typeof key != 'undefined', 'should have at least one data sheet');
                return getDataSheet(t[key].data);
            })
            .then(data => {
                console.log(data.length);
                console.log(data);
                assert(data.length > 0, 'should have data');
            })
    })
    
    
})


google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');

function getTemplateProperties(value, properties, templates) {
    var returnObj = {};
    var promise;
    if(typeof properties[value + '-data'] != 'undefined') {
        promise = Promise.resolve(properties[value + '-data']);
    } else {
        promise = getDataSheet(templates[value].data);
    }
    return promise
        .then(data => {
            properties[value + '-data'] = data;
            returnObj[value + '-data'] = data;
            var templatePromise;
            if(typeof properties[value] != 'undefined') {
                templatePromise = Promise.resolve(properties[value]);
            } else {
                templatePromise = getSections(value, properties, templates)
                    .then(rows => renderRows(value, rows, properties));
            }
            return templatePromise;
        })
        .then(template => {
            properties[value] = template;
            returnObj[value] = template;
            return returnObj;
        });
}

function getRows(cells) {
    // turn cells in to rows, TODO: can getCells options do this for me?
    var rows = cells.reduce((arr, c) => {
        if(typeof arr[c.row - 1] == 'undefined') arr[c.row - 1] = [];
        arr[c.row - 1][c.col - 1] = c.value;
        return arr;
    }, []);
    return rows;
}

// must do this up front so we can process all data
function getSections(layoutTitle, properties, templates) {
    var worksheet = templates[layoutTitle].template;
    if(!worksheet) {
        return Promise.resolve(false);
    }
    return util.promisify(worksheet.getCells)({
        'min-row': 1,
        'max-row': 100,
        'return-empty': false
    })
        .then(cells => getRows(cells))
        .then(rows => {
            var sections = rows.reduce((obj, row, i) => {
                var subsections = row.reduce((obj, c, j) => {
                    var value = row[j + 1];
                    if(c === ':section' && typeof templates[value] != 'undefined') {
                        obj[value] = getTemplateProperties(value, properties, templates);
                    }
                    return obj;
                }, obj);
                return obj;
            }, {});
            return Promise.all(Object.values(sections)).then(() => rows);
        })
}

module.exports = getTemplateProperties;

google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var importer = require('../Core');
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    var dataClasses = dataKeys
        .filter(k => ('\\{\\{' + k + '\\}\\}', 'ig').match(c))
        .map(k => 'val-' + k)
        .join(' ')
    return dataClasses;
}

function renderRows(layoutTitle, rows, properties) {
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                var value = row[j + 1];
                if(c.substr(1).toLowerCase() === 'section') {
                    arr[arr.length] = `
<div class="section-${value}">
${properties[value]}
</div>
`;
                } else {
                    properties[c.substr(1)] = value;
                }
            // render markdown content if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1]
                      && row[j - 1].substr(0, 1) !== ':') {
                var dataClasses = getDataClasses(c, properties[layoutTitle + '-data']);

                var mdHtml = md.render(c);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(mdHtml.replace(/<\/?p>|\s+/ig, '') == c.replace(/<\/?p>|\s+/ig, '')) {
                    mdHtml = c;
                }
                arr[arr.length] = `
<div class="${dataClasses} cell-${j}">
${mdHtml}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            var rowTemplate = `
<div class="${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
            arr[arr.length] = rowTemplate;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;

test google sheet template?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var renderRows = importer.import('google sheet layout template');
var getTemplateProperties = importer.import('google sheet template properties');
var getDataSheet = importer.import('google sheet array objects');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('get html from template', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getDataSheet(templates[key].data);
            })
            .then(data => {
                assert(typeof key != 'undefined', 'should have at least one template');
                properties[key + '-data'] = data;
                return getTemplateProperties(key, properties, templates);
            })
            .then(() => {
                console.log(properties[key]);
                console.log(properties);
                assert(properties[key].length > 0, 'should have a page called template');
            })
    })
    
    
})


output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function wrapTemplate(layoutTitle, html, properties) {
    // TODO: automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
<link rel="stylesheet" media="screen" href="/illuminati-style.css">
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
    background-image: url({{{banner}}});
}
</style>
<!-- Load Stripe.js on your website. -->
<script src="https://js.stripe.com/v3"></script>
<script>
var stripe = Stripe('pk_live_4LdeNXQQ1sm3SECaJRr5lMg9000RQ4FXDa');

function checkout(evt) {
    var sku = evt.value;
    // When the customer clicks on the button, redirect
    // them to Checkout.
    stripe.redirectToCheckout({
      items: [{sku: sku, quantity: 1}],
      successUrl: window.location.protocol + '//' + window.location.host + '/success',
      cancelUrl: window.location.protocol + '//' + window.location.host + '/canceled',
    })
    .then(function (result) {
      if (result.error) {
        // error, display the localized error message to your customer.
        var displayError = document.getElementById('error-message');
        displayError.textContent = result.error.message;
      }
    });
}
</script>
</head>
<body class="${layoutTitle} {{class}}">
${html}
<div id="error-message"></div>
</body>
</html>`;
    Mustache.parse(pageHtml);
    return Mustache.render(pageHtml, properties);
}

module.exports = wrapTemplate;


test output sheet template?


In [ ]:
var fs = require('fs');
var path = require('path');
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var getTemplateProperties = importer.import('google sheet template properties');
var wrapTemplate = importer.import('output google sheet template');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('output rendered html', () => {
    
    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getTemplateProperties(key, properties, templates)
                    .then(() => wrapTemplate(key, properties[key], properties))
            })
            .then(page => {
                assert(page.length > 0, 'should have a page');
            })
    })
    
    
})


collect google sheets resources?


In [ ]:
var fs = require('fs');
var path = require('path');
var {JSDOM, XPathResult} = require('jsdom');
var importer = require('../Core');
var findSimilarFile = importer.import('find similar file');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var getArrayXPath = importer.import('get xpath array');
var PROJECT_OUTPUT = path.resolve(path.join(__dirname, "../.output"));
var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

// detect links and write out every part of the site
function collectTemplateResources(layoutTitle, page, properties, templates, rendered) {
    if(!rendered) {
        rendered = [];
    }
    var filename = path.join(PROJECT_OUTPUT, layoutTitle + '.html');
    fs.writeFileSync(filename, page);
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));
    
    var backgrounds = regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var paths = [];
    // copy resource images to output directory
    var searches = imgs.concat(styles).concat(backgrounds)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':') && rendered.indexOf(cur) === -1)
        .map(i => {
            rendered[rendered.length] = i;
            return findSimilarFile(path.basename(i), DOWNLOAD_DIR)
                .then(f => {
                    if(f) {
                        fs.copyFileSync(
                            path.join(DOWNLOAD_DIR, f),
                            path.join(PROJECT_OUTPUT, path.basename(f)));
                    }
                });
        });
    
    var linked = links
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':'))
        .map(l => {
            var urlParts = l.split('/')
            var key = urlParts[1];
            var fileName = key;
            if(rendered.indexOf(l) > -1 || typeof templates[key] == 'undefined') {
                return Promise.resolve(false);
            }
            
            console.log('Rendering ' + l);
            rendered[rendered.length] = l;
            return getTemplateProperties(key, properties, templates)
                .then(() => {
                    // if rendering a page has url parts
                    if(urlParts.length > 2) {
                        if(!fs.existsSync(path.join(PROJECT_OUTPUT, urlParts[1]))) {
                            fs.mkdirSync(path.join(PROJECT_OUTPUT, urlParts[1]));
                        }

                        // use the third argument to find the correct row in the data set
                        var rowData = properties[key + '-data']
                            .filter(row => row['link'] == l || row['url'] == l)[0];
                        Object.assign(properties, rowData);
                        fileName = l.substr(1);
                    }
                    return wrapTemplate(key, properties[key], properties)
                })
                .then(page => collectTemplateResources(fileName, page, properties, templates, rendered))
        });
    return Promise.all(searches)
        .then(() => Promise.all(linked));
}

module.exports = collectTemplateResources;

test google sheets resources?


In [ ]:
var assert = require('assert');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var docsId = '113mVIumItArQ_oXpfDRXP-2Kw2ms4t48oPJ68_p5P8k';

describe('output template resources', () => {

    it('should process at least one template', () => {
        var templates, key, properties = {};
        return getTemplates(docsId)
            .then(t => {
                templates = t;
                key = Object.keys(templates).filter(k => templates[k].template)[0];
                return getTemplateProperties(key, properties, templates)
                    .then(() => wrapTemplate(key, properties[key], properties));
            })
            .then(page => {
                // TODO: move this in to a new function for detecting links and writing every part of site
                assert(page.length > 0, 'should have a page');
                return collectTemplateResources(key, page, properties, templates);
            })
    }).timeout(60000)
    
    
})
